# Dash visualisation of FireRP and Precipitation

In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [2]:
from platform import python_version
import sys
print('Python: ' + python_version()) # Python: 3.12.
print ('dash: ' + sys.modules["dash"].__version__) # dash: 2.17.0
print ('plotly: ' + sys.modules["plotly"].__version__) # plotly: 5.22.0
print('pandas: ' + pd.__version__) # pandas: 2.2.1
print('numpy: ' + np.__version__) # numpy: 1.26.4

Python: 3.12.2
dash: 2.17.0
plotly: 5.22.0
pandas: 2.2.1
numpy: 1.26.4


## Data import
The Fire Reactive Power is the log transformed binned summary of the MWIR data

In [3]:
datacube_precip = np.load("./data/WaterPrecip_datacube.npy")
datacube_FireRF = np.load("./data/FireRP_log_datacube_MWIR.npy")

time_precip = np.load("./data/WaterPrecip_time.npy")
time_FireRF = np.load("./data/FireRP_time.npy")

latitude = np.load("./data/WaterPrecip_latitude.npy")
longitude = np.load("./data/WaterPrecip_longitude.npy")

In [4]:
# Trim the Pricipitation dataset to the same size as the FireRP dataset
datacube_precip = datacube_precip[np.argwhere(time_precip == time_FireRF[0])[0][0]:np.argwhere(time_precip == time_FireRF[-1])[0][0]+1]

## Create a geojson grid of 1° Latitude x 1° Longitude
### geojson file format:
```python
# Feature collection object
{'type': 'FeatureCollection', 
 # List of features
 'features': [
     {'type': 'Feature',
      # supported geometry types: Point, LineString, Polygon, MultiPoint, MultiLineString, and MultiPolygon.
      'geometry':
          {'type': 'Polygon', 
            # List of coordinates
           'coordinates': 
               [[[lon_1, lat_1], [lon_2, lat_2], ....]]}
      # Dictionary of property key values sets
      'properties': {key: value},
      'id': x},
     .... # further features
 ]
}
```

In [5]:
# create an empty feature collection
geojson = {'type': 'FeatureCollection', 
                'features': []}
# create a feature for each 1° Latitude x 1° Longitude square
# 0° x 0° are at the coner of the grid
for x in range(180):
    for y in range(360):
        # Add the corners of the 1x1 square to a geometry object
        # The lat and lon are added as 90N -90S -180W 180E format
        temp_geometry = {'type': 'Polygon',
                 'coordinates': [[[y-180, x-90],
                                 [y-179, x-90],
                                 [y-179, x-89],
                                 [y-180, x-89]]]}
        # Add the geographical coordinates to the feature
        temp_features = {'type': 'Feature', 
                         'geometry': temp_geometry,
                         # Dictionary of property key values sets
                         # an easy readable string is choosen as feature name
                         'properties': {'location': "lat: " + str(x) + " lon: " + str(y)},
                         # unique int id for each feature
                         'id': x*360+y}
        # add feature to feature collection
        geojson["features"].append(temp_features)

## Create the dataframes from the datacubes

In [6]:
temp = pd.DataFrame({
    "Date":  time_FireRF,
    "FireRP": list(datacube_FireRF),
    "Precip": list(datacube_precip)
})

## Sep Date
temp.loc[:,'Year'] = temp.loc[:,'Date'].map(lambda x: int(str(x)[:4]))
temp.loc[:,'Month'] = temp.loc[:,'Date'].map(lambda x: int(str(x)[4:6]))
temp.loc[:,'Day'] = temp.loc[:,'Date'].map(lambda x: int(str(x)[6:8]))

In [7]:
# clean and normalize the data to [0,1]
datacube_precip[datacube_precip == -99999] = 0
datacube_precip = datacube_precip - datacube_precip.min()
datacube_FireRF = datacube_FireRF - datacube_FireRF.min()
datacube_precip = datacube_precip/100
datacube_FireRF = datacube_FireRF/datacube_FireRF.max()
temp_2 = pd.DataFrame({
    "location":  [(x,y) for x in range(180) for y in range(360)],
    "FireRP": list(datacube_FireRF.reshape(-1, 180*360).T),
    "Precip": list(datacube_precip.reshape(-1, 180*360).T)
})
## Sep location
temp_2.loc[:,'lat'] = temp_2.loc[:,'location'].map(lambda x: x[0])
temp_2.loc[:,'lon'] = temp_2.loc[:,'location'].map(lambda x: x[1])
# rename the column location to match the geojson feature name
temp_2["location"] = temp_2.apply(lambda row: "lat: " + str(row["lat"]) + " lon: " +  str(row["lon"]), axis=1)

## Create Dashboard

In [8]:
global_figure_styles = dict(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor= '#e4edf4')

In [9]:
# Create a Dash application instance
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    # Header for the application
    html.H1(children='Data Visualisation FireRP and Precipitation', style={'textAlign':'center'}),
    # Radio buttons to select dataset (FireRP or Precipitation)
    dcc.RadioItems(
        id='dataset', 
        options=["FireRP", "Precipitation"],
        value="FireRP",
        inline=True
    ),
    # Slider to select the month
    dcc.Slider(
        0, 11, step=None,
        marks={
            0: '2020-03',
            1: '2020-04',
            2: '2020-05',
            3: '2020-06',
            4: '2020-07',
            5: '2020-08',
            6: '2020-09',
            7: '2020-10',
            8: '2020-11',
            9: '2020-12',
            10: '2021-01',
            11: '2021-02',
        },
        value = 0,
        id='Month',
        included=False
    ),
    # Slider to select the day
    dcc.Slider(
        min = 1,
        max = 31,
        step = 1,
        value = 1,
        id='Day',
        included=False
    ),
    # Graph component to display the main plot
    dcc.Graph(id="graph"),
    # Graph component to display the time series plot
    dcc.Graph(id="graph_time"),
])

# Callback to set the maximum day value based on the selected month
@callback(
    Output('Day', 'max'),
    Input('Month', 'value')
)
def set_day_value(month):
    # Match the month value to return the corresponding maximum number of days
    match month:
        case 0: return 31
        case 1: return 30
        case 2: return 31
        case 3: return 30
        case 4: return 31
        case 5: return 31
        case 6: return 30
        case 7: return 31
        case 8: return 30
        case 9: return 31
        case 10: return 31
        case 11: return 27
        case _: return 31

# Callback to update the main plot based on the selected dataset, month, and day
@callback(
    Output("graph", "figure"), 
    Input('dataset', 'value'),
    Input('Month', 'value'),
    Input('Day', 'value')
)
def display_choropleth(dataset, month, day):
    # Adjust the month value for indexing
    month = (month+3)%13 + int(month/10)
    # Prepare the data based on the selected dataset
    if dataset == "FireRP":
        temp_arr = pd.DataFrame(*temp.loc[(temp.loc[:,'Month'] == month) & (temp.loc[:,'Day'] == day),'FireRP']).stack().reset_index()
        temp_scale = [(0, "white"), (0.5, "orange"), (1, "red")]
    else:
        temp_arr = pd.DataFrame(*temp.loc[(temp.loc[:,'Month'] == month) & (temp.loc[:,'Day'] == day),'Precip']).stack().reset_index()
        temp_scale = [(0, "white"), (0.5, "blue"), (1, "navy")]
    # Create a new column combining latitude and longitude information
    temp_arr["location"] = temp_arr.apply(lambda row: "lat: " + str(int(row["level_0"])) + " lon: " +  str(int(row["level_1"])), axis=1)
    temp_arr.rename({0: "value"}, axis='columns', inplace=True)
    # Create a choropleth map
    fig = px.choropleth_mapbox(temp_arr, geojson=geojson, color="value", opacity=.2,
                               color_continuous_scale=temp_scale,
                               locations="location", featureidkey="properties.location", 
                               center={"lat": 50, "lon": 10}, 
                               mapbox_style="carto-positron", zoom=3)
    fig.update_layout(**global_figure_styles,
                      clickmode='event+select')
    return fig

# Callback to update the time series plot based on the selected point in the main plot
@callback(
    Output("graph_time", "figure"), 
    Input("graph", "clickData"),
)
def display_line(click):
    itemp = 0
    # Check if a point in the main plot is clicked
    if click is not None:
        itemp = click['points'][0]['pointNumber']
    
    # Create a line plot for the selected point
    fig = go.Figure(data=px.line(pd.DataFrame({'Precip': temp_2.loc[itemp, "Precip"],'FireRP': temp_2.loc[itemp, "FireRP"]}).stack().reset_index(),
                                    x = 'level_0', y = 0, color = 'level_1'))
    fig.update_layout(**global_figure_styles, 
                      height=300)
    return fig


In [10]:
if __name__ == '__main__':
    app.run(jupyter_height=1000, debug=True, port = 8050)